In [0]:
import numpy as np
import tensorflow as tf

def removeChars(remove_chars, text):
  for rchar in remove_chars:
    text = text.replace(rchar, ' ')
  return text

def removeWords(remove_words, text):
  text_words = text.split()
  text = [word for word in text_words if word not in remove_words]
  return ' '.join(text)

def replaceChars(chars, new_char, text):
  for char in chars:
    text = text.replace(char, new_char)
  return text

def to_one_hot(data_point_idx, vocab_size):
  temp = np.zeros(vocab_size)
  temp[data_point_idx] = 1
  return temp

def word2vec(word, int2vec):
  if(word not in word2int):
    return 0
  else:
    return int2vec[word2int[word]]

def cosine_similarity(vec1, vec2):
  return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))

def euclidean_dist(vec1, vec2):
  return np.sqrt(np.sum((vec1-vec2)**2))

def find_closest_euclidean_dist(word_index, int2vec):
  min_dist = 10000 # to act like positive infinity
  min_index = -1
  query_vector = int2vec[word_index]
  for index, vector in enumerate(int2vec):
    if euclidean_dist(vector, query_vector) < min_dist and not np.array_equal(vector, query_vector):
      min_dist = euclidean_dist(vector, query_vector)
      min_index = index
  return min_index, min_dist

def find_closest_cosine_similarity(word_index, int2vec):
  max_dist = -10000 # to act like positive infinity
  max_index = -1
  query_vector = int2vec[word_index]
  for index, vector in enumerate(int2vec):
    if cosine_similarity(vector, query_vector) > max_dist and not np.array_equal(vector, query_vector):
      max_dist = cosine_similarity(vector, query_vector)
      max_index = index
  return max_index, max_dist

In [0]:
corpus = [
    '¿Qué es un lenguaje de programación?',
    '¿Qué es un compilador?',
    '¿Qué lenguaje de programación enseñan?',
    '¿Importa en qué línea pongo mi código?',
    '¿Cómo imprimo mis programas?',
    '¿Cuál es la diferencia entre === y ==?',
    '¿Cuándo uso un bucle for y cuándo un while?',
    '¿Para qué necesito variables, no es más fácil simplemente ingresar el número?',
    '¿Cómo aprendo a programar?',
    '¿Qué es la programación orientada a objetos?',
    '¿Qué editor utilizas?',
    '¿Qué es un código fuente?',
    '¿Qué es un código máquina?',
    '¿Qué es un archivo ejecutable?',
    '¿En qué consiste la recursividad?']

# Sentences
sentences = [sentence.lower() for sentence in corpus]
sentences = [removeChars(['¿','?',','], sentence) for sentence in sentences]
sentences = [removeWords(['es','un','una','qué','el','la','de', 'al', 'a', 'no', 'cómo', 'cuál', 'para', 'cuándo', 'en', 'mi', 'mis', 'más', 'entre', 'y'], sentence) for sentence in sentences]
sentences = [sentence.split() for sentence in sentences]
print(sentences)

# Words
words = []
for sentence in sentences:
  words.extend(sentence)
words = set(words)
print(words)

# Sentences
word2int = {}
int2word = {}

vocab_size = len(words)
for word_idx, word in enumerate(words):
  word2int[word] = word_idx
  int2word[word_idx] = word

[['lenguaje', 'programación'], ['compilador'], ['lenguaje', 'programación', 'enseñan'], ['importa', 'línea', 'pongo', 'código'], ['imprimo', 'programas'], ['diferencia', '===', '=='], ['uso', 'bucle', 'for', 'while'], ['necesito', 'variables', 'fácil', 'simplemente', 'ingresar', 'número'], ['aprendo', 'programar'], ['programación', 'orientada', 'objetos'], ['editor', 'utilizas'], ['código', 'fuente'], ['código', 'máquina'], ['archivo', 'ejecutable'], ['consiste', 'recursividad']]
{'fácil', 'línea', 'fuente', 'bucle', 'variables', 'programas', 'archivo', 'ejecutable', 'número', '===', 'importa', 'recursividad', 'código', 'compilador', 'utilizas', 'diferencia', 'programar', 'objetos', 'while', 'ingresar', 'uso', 'enseñan', 'simplemente', 'imprimo', 'for', 'orientada', '==', 'pongo', 'aprendo', 'máquina', 'lenguaje', 'editor', 'consiste', 'necesito', 'programación'}


In [0]:
# Data
data = []
WINDOW_SIZE = 2

for sentence in sentences:
  for word_idx, word in enumerate(sentence):
    for word_neighbor in sentence[max(0, word_idx - WINDOW_SIZE) : min(word_idx + WINDOW_SIZE , len(sentence) - 1) + 1]:
      if word_neighbor != word:
        data.append([word, word_neighbor])
print(data)

x_train = []
y_train = []

for data_word in data:
  x_train.append(to_one_hot(word2int[data_word[0]], vocab_size))
  y_train.append(to_one_hot(word2int[data_word[1]], vocab_size))  
  
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
print(x_train, y_train)
print(x_train.shape, y_train.shape)

[['lenguaje', 'programación'], ['programación', 'lenguaje'], ['lenguaje', 'programación'], ['lenguaje', 'enseñan'], ['programación', 'lenguaje'], ['programación', 'enseñan'], ['enseñan', 'lenguaje'], ['enseñan', 'programación'], ['importa', 'línea'], ['importa', 'pongo'], ['línea', 'importa'], ['línea', 'pongo'], ['línea', 'código'], ['pongo', 'importa'], ['pongo', 'línea'], ['pongo', 'código'], ['código', 'línea'], ['código', 'pongo'], ['imprimo', 'programas'], ['programas', 'imprimo'], ['diferencia', '==='], ['diferencia', '=='], ['===', 'diferencia'], ['===', '=='], ['==', 'diferencia'], ['==', '==='], ['uso', 'bucle'], ['uso', 'for'], ['bucle', 'uso'], ['bucle', 'for'], ['bucle', 'while'], ['for', 'uso'], ['for', 'bucle'], ['for', 'while'], ['while', 'bucle'], ['while', 'for'], ['necesito', 'variables'], ['necesito', 'fácil'], ['variables', 'necesito'], ['variables', 'fácil'], ['variables', 'simplemente'], ['fácil', 'necesito'], ['fácil', 'variables'], ['fácil', 'simplemente'], ['f

In [0]:
# Model
EMBEDDING_DIM = 10

x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y = tf.placeholder(tf.float32, shape=(None, vocab_size))

W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
hidden_representation = tf.add(tf.matmul(x,W1), b1)

W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2))

Loss in 0 is: 9.065935134887695
Loss in 1000 is: 3.3575074672698975
Loss in 2000 is: 2.2417328357696533
Loss in 3000 is: 1.6735219955444336
Loss in 4000 is: 1.372108817100525
Loss in 5000 is: 1.2084364891052246
Loss in 6000 is: 1.110062599182129
Loss in 7000 is: 1.0458849668502808
Loss in 8000 is: 1.0018970966339111
Loss in 9000 is: 0.970647394657135
Loss in 10000 is: 0.9477781653404236
Loss in 11000 is: 0.9306064248085022
Loss in 12000 is: 0.9174143671989441
Loss in 13000 is: 0.9070685505867004
Loss in 14000 is: 0.8988023400306702
Loss in 15000 is: 0.8920857906341553
Loss in 16000 is: 0.8865461945533752
Loss in 17000 is: 0.8819155097007751
Loss in 18000 is: 0.8779979944229126
Loss in 19000 is: 0.8746480941772461


In [0]:
# Int to Vector
int2vec = sess.run(W1 + b1)
# print(int2vec)

print(word2vec('código', int2vec))

[-1.1042812  2.9002814  2.3381248  1.0156506  0.7340162 -1.246618
  0.2800207 -1.0904531  1.4586371 -1.2897786]


In [0]:
# Word Similarity
print('Cosine Similarity')
for word in words:
  idx, dis = find_closest_cosine_similarity(word2int[word], int2vec)
  print('{} -> {} ({})'.format(word, int2word[idx], dis))
  
# print('\nEuclidean Distance')
# for word in words:
#   idx, dis = find_closest_euclidean_dist(word2int[word], int2vec)
#   print('{} -> {} ({})'.format(word, int2word[idx], dis))

Cosine Similarity
fácil -> variables (0.6715167760848999)
línea -> importa (0.8054413795471191)
fuente -> pongo (0.6003442406654358)
bucle -> for (0.6461241841316223)
variables -> imprimo (0.6813637018203735)
programas -> pongo (0.6864524483680725)
archivo -> objetos (0.25295379757881165)
ejecutable -> bucle (0.5732983350753784)
número -> línea (0.6013498902320862)
=== -> == (0.7513993382453918)
importa -> línea (0.8054413795471191)
recursividad -> programar (0.5482661724090576)
código -> simplemente (0.583342969417572)
compilador -> fuente (0.457430362701416)
utilizas -> objetos (0.7300836443901062)
diferencia -> objetos (0.6978047490119934)
programar -> consiste (0.6808789968490601)
objetos -> utilizas (0.7300836443901062)
while -> uso (0.7592750191688538)
ingresar -> simplemente (0.7138216495513916)
uso -> while (0.7592750191688538)
enseñan -> lenguaje (0.7668620347976685)
simplemente -> necesito (0.7260639667510986)
imprimo -> variables (0.6813637018203735)
for -> bucle (0.64612418

In [0]:
# Sentences
sentences = [sentence.lower() for sentence in corpus]
sentences = [removeChars(['¿','?',','], sentence) for sentence in sentences]
sentences = [removeWords(['es','un','una','qué','el','la','de', 'al', 'a', 'no', 'cómo', 'cuál', 'para', 'cuándo', 'en', 'mi', 'mis', 'más', 'entre', 'y'], sentence) for sentence in sentences]

def sentence2vec(sentence):
  words = sentence.split()
  count = len(words)
  meaning = 0
  for word in words:
    meaning += word2vec(word, int2vec)
  meaning /= count
  
  return meaning

def closestSentence(sentence, sentences, include_itself=True, show_all=False):
  max_similarity = 0
  best_sentence = ''
  for other_sentence in sentences:
    try:
      similarity = cosine_similarity(sentence2vec(sentence), sentence2vec(other_sentence))
      if max_similarity < similarity:
        if include_itself or sentence!=other_sentence:
          max_similarity = similarity
          best_sentence = other_sentence
    except:
      pass
  return (max_similarity, best_sentence if max_similarity != 0 else 'No entendí tu pregunta')
#   return 0, 'No entendí tu pregunta'

for sentence in sentences:
  print(sentence + ' -> ' + closestSentence(sentence, sentences, include_itself=False)[1])

lenguaje programación -> lenguaje programación enseñan
compilador -> aprendo programar
lenguaje programación enseñan -> lenguaje programación
importa línea pongo código -> aprendo programar
imprimo programas -> importa línea pongo código
diferencia === == -> programación orientada objetos
uso bucle for while -> editor utilizas
necesito variables fácil simplemente ingresar número -> imprimo programas
aprendo programar -> consiste recursividad
programación orientada objetos -> lenguaje programación
editor utilizas -> programación orientada objetos
código fuente -> código máquina
código máquina -> código fuente
archivo ejecutable -> uso bucle for while
consiste recursividad -> aprendo programar


In [0]:
my_question = 'para que necesito variables?'
my_question = my_question.lower()
my_question = removeChars(['¿','?',','], my_question)
my_question = removeWords(['es','un','una','qué','el','la','de', 'al', 'a', 'no', 'cómo', 'cuál', 'para', 'cuándo', 'en', 'mi', 'mis', 'más', 'entre', 'y'], my_question)

max_similarity, best_sentence = closestSentence(my_question, sentences)
print('\nBest match: {} ({})'.format(best_sentence, max_similarity))


Best match: necesito variables fácil simplemente ingresar número (0.9196069836616516)
